#ドライブにマウント

In [1]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


#ライブラリインストール

In [2]:
!pip install -q transformers==4.4.2 pytorch_lightning==1.2.1 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.2/814.2 KB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 67.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 KB 66.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
#ここには配布したファイルのDIRを
MODEL_DIR = "/content/drive/MyDrive/murata-lab/summary/t5"
#事前学習済みのやつ
PRETRAINED_MODEL_NAME = "sonoisa/t5-base-japanese"

#入力文字の正規化の関数

In [4]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

In [5]:
def normalize_text(text):
    assert "\n" not in text and "\r" not in text
    text = text.replace("\t", " ")
    text = text.strip()
    text = normalize_neologd(text)
    text = text.lower()
    return text

In [6]:
#20文字以上の文章を入力
body = """
今回の攻撃はパキスタンの政府が支援したものだと
"""

In [7]:
MAX_SOURCE_LENGTH = 25   # 入力される記事本文の最大トークン数
MAX_TARGET_LENGTH = 20  # 生成されるタイトルの最大トークン数
from transformers import T5Tokenizer,T5Config, T5Model,T5ForConditionalGeneration

# 事前学習済みモデル
tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL_NAME, is_fast=True)
#配布したファイル
trained_model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)

#入力文章の正規化関数
def preprocess_body(text):
    return normalize_text(text.replace("\n", " "))

# 推論モード設定
trained_model.eval()

# 前処理とトークナイズを行う
inputs = [preprocess_body(body)]
#inputをトークナイズ
batch = tokenizer.batch_encode_plus(
    inputs, max_length=MAX_SOURCE_LENGTH, truncation=True, 
    padding="longest", return_tensors="pt")
print(batch)
input_ids = batch['input_ids']
input_mask = batch['attention_mask']
# 生成処理を行う
outputs = trained_model.generate(
    input_ids=input_ids, attention_mask=input_mask, 
    max_length=MAX_TARGET_LENGTH,
    temperature=1.0,          # 生成にランダム性を入れる温度パラメータ
    num_beams=10,             # ビームサーチの探索幅
    diversity_penalty=1.0,    # 生成結果の多様性を生み出すためのペナルティ
    num_beam_groups=10,       # ビームサーチのグループ数
    num_return_sequences=1,  
    repetition_penalty=1.5,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
)

# 生成されたトークン列を文字列に変換する
generated_titles = [tokenizer.decode(ids, skip_special_tokens=True, 
                                     clean_up_tokenization_spaces=False) 
                    for ids in outputs]

# 生成されたタイトルを表示する
for i, title in enumerate(generated_titles):
    print(title)
    more_21_chr=title

Downloading:   0%|          | 0.00/804k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

{'input_ids': tensor([[    5, 12360,   649,     7, 11538,     6,  9196,  1264,  2376,  1863,
             1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
今回の攻撃はパキスタンの政府が支援したものだと


#Mecab
##今は辞書古いけどipa-dic neologd使うほうがよさげ

In [24]:
! pip install mecab-python3 unidic-lite
import MeCab
output_text=""

wakatilist=MeCab.Tagger("-Owakati").parse(more_21_chr).split()
print(wakatilist)
for i in range(len(wakatilist)):
  if len(output_text)+len(wakatilist[i])>=21:
    break
  output_text=output_text+wakatilist[i]
  
print(output_text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
['今回', 'の', '攻撃', 'は', 'パキスタン', 'の', '政府', 'が', '支援', 'し', 'た', 'もの', 'だ', 'と']
今回
今回の
今回の攻撃
今回の攻撃は
今回の攻撃はパキスタン
今回の攻撃はパキスタンの
今回の攻撃はパキスタンの政府
今回の攻撃はパキスタンの政府が
今回の攻撃はパキスタンの政府が支援
今回の攻撃はパキスタンの政府が支援し
今回の攻撃はパキスタンの政府が支援した
今回の攻撃はパキスタンの政府が支援した
